In [732]:
import finnhub
from django.conf import settings
import numpy as np
import pandas as pd
import datetime as dt
import requests
import environ
import os
from pprint import pprint

In [733]:
env = environ.Env()

In [734]:
finnhub_client = finnhub.Client(api_key=env('FINNHUB_KEY'))

In [735]:
# configures max columns displayed in df 
pd.set_option('display.max_columns', 85)
# configures max rows displayed in df (using tab)
pd.options.display.max_rows = 85

In [736]:
option_orders = pd.read_csv('./data/Webull_Orders_Records_Options.csv')
stock_orders = pd.read_csv('./data/Webull_Orders_Records.csv')

In [737]:
# GRAB PRICE DATA FOR SYMBOL
def fh_get_stock_candles_data(symbol, start=1540988249, end=1591852249):

    # Stock df
    res = finnhub_client.stock_candles(symbol, 'D', start, end)  # 1540988249 1591852249

    #Convert to Pandas Dataframe
    df = pd.DataFrame(res)

    df.rename(columns={
        'c': 'close',
        'h': 'high',
        'l': 'low', 
        'o': 'open', 
        't': 'time',
        'v': 'volume'
    }, inplace=True)

    df['date'] = pd.to_datetime(df['time'], unit='s')
    df = df[['date', 'time', 'open', 'high', 'low', 'close', 'volume']]

    df['delta open'] = df['open'].diff()
    df['delta high'] = df['high'].diff()
    df['delta low'] = df['low'].diff()
    df['delta close'] = df['close'].diff()
    df['delta volume'] = df['volume'].diff()

    # SMA 3 - CLOSE
    for i in range(0, df.shape[0]-2):
        df.loc[df.index[i+2],'sma3 close'] = np.round(((df.iloc[i,4]+ df.iloc[i+1,4] +df.iloc[i+2,4])/3),1)

    # SMA 3 - VOLUME
    for i in range(0, df.shape[0]-2):
        df.loc[df.index[i+2],'sma3 volume'] = np.round(((df.iloc[i,5]+ df.iloc[i+1,5] +df.iloc[i+2,5])/3),1)

    df['sma3 close'] = df['close'].diff()
    df['sma3 volume'] = df['volume'].diff()
    df['sma50 close'] = df['close'].rolling(50).mean()
    df['sma50 volume'] = df['volume'].rolling(50).mean()
    df['sma200 close'] = df['close'].rolling(200).mean()
    df['sma200 volume'] = df['volume'].rolling(200).mean()
    df['sma50 cross close above'] = (df['sma50 close'] > df['close'])
    df['sma50 cross close below'] = (df['sma50 close'] < df['close'])
    df['sma50 cross volume above'] = (df['sma50 volume'] > df['volume'])
    df['sma50 cross volume below'] = (df['sma50 volume'] < df['volume'])
    df['sma200 cross close above'] = (df['sma200 close'] > df['close'])
    df['sma200 cross close below'] = (df['sma200 close'] < df['close'])
    df['sma200 cross volume above'] = (df['sma200 volume'] > df['volume'])
    df['sma200 cross volume below'] = (df['sma200 volume'] < df['volume'])

    # COMPARISON BOOLEAN COLUMN THAT COMPARES PREVIOUS ROWS VALUE TO CURRENT
    df['position'] = np.where((df['volume'] > df['volume'].shift(1)) & ((df['close'] >= df['close'].shift(1)) & (df['open'] <= df['open'].shift(1))),"UP","DOWN")

    return df

In [738]:
def fh_get_company_profile(symbol):
    
    # COMPANY PROFILE
    res = finnhub_client.company_profile2(symbol='AAPL')

    return res

In [739]:
candles = fh_get_stock_candles_data('TSLA')
profile = fh_get_company_profile('TSLA')

In [740]:
pprint(profile)

{'country': 'US',
 'currency': 'USD',
 'exchange': 'NASDAQ NMS - GLOBAL MARKET',
 'finnhubIndustry': 'Technology',
 'ipo': '1980-12-12',
 'logo': 'https://finnhub.io/api/logo?symbol=AAPL',
 'marketCapitalization': 2600759,
 'name': 'Apple Inc',
 'phone': '14089961010.0',
 'shareOutstanding': 16334.37,
 'ticker': 'AAPL',
 'weburl': 'https://www.apple.com/'}


In [741]:
def process_option_name(text):
    symbol, date, time, zone, order, price = [x.strip() for x in text.split(' ')]
    time = ' '.join([date, time, zone])
    return pd.Series([time, symbol, order, price])

In [742]:
option_orders[['Contract Exp', 'Symbol', 'Type', 'Price']] = option_orders.Name.apply(process_option_name)

In [743]:
filled_filter = (option_orders['Status'] == 'Filled')
option_orders = option_orders[filled_filter]
option_orders = option_orders.iloc[::-1]
option_orders['Price'] = (option_orders['Price'].astype(str).str[1:]).astype(float)

In [744]:
option_orders = option_orders[['Placed Time', 'Filled Time', 'Symbol', 'Price', 'Avg Price', 'Type', 'Side', 'Filled', 'Total Qty', 'Time-in-Force']]

In [745]:
option_orders

,Placed Time,Filled Time,Symbol,Price,Avg Price,Type,Side,Filled,Total Qty,Time-in-Force
135,06/14/2021 09:42:10 EDT,06/14/2021 09:42:15 EDT,Z,120.0,1.2000,Call,Buy,1,1,DAY
134,06/14/2021 09:51:18 EDT,06/14/2021 09:52:46 EDT,Z,120.0,0.9300,Call,Buy,1,1,DAY
133,06/14/2021 10:21:02 EDT,06/14/2021 10:23:27 EDT,BLUE,45.0,1.3500,Call,Buy,1,1,DAY
131,06/14/2021 11:11:17 EDT,06/14/2021 11:19:17 EDT,IVR,5.0,0.1700,Call,Buy,3,3,DAY
130,06/14/2021 15:54:24 EDT,06/14/2021 15:54:24 EDT,Z,120.0,0.6100,Call,Sell,2,2,DAY
128,06/15/2021 09:37:12 EDT,06/15/2021 10:13:00 EDT,TR,40.0,0.7000,Call,Buy,3,3,DAY
127,06/15/2021 09:50:13 EDT,06/15/2021 10:05:36 EDT,INTC,62.0,0.1300,Call,Buy,2,2,DAY
125,06/15/2021 10:31:44 EDT,06/15/2021 10:57:09 EDT,BLUE,45.0,0.6000,Call,Sell,1,1,DAY
123,06/15/2021 12:48:10 EDT,06/15/2021 12:48:19 EDT,INTC,62.0,0.1800,Call,Sell,2,2,DAY
121,06/15/2021 12:50:32 EDT,06/15/2021 13:47:42 EDT,TR,40.0,0.5000,Call,Buy,2,2,DAY


In [746]:
option_symbols = option_orders['Symbol'].unique()

In [747]:
def update_cost_quantity(df):
    df.loc[df.Side == 'Buy', "Total Cost"] = df['Total Cost'] * -1
    df.loc[df.Side == 'Sell', "Total Qty"] = df['Total Qty'] * -1
    return df

In [748]:
filled_filter = (stock_orders['Status'] == 'Filled')
stock_orders = stock_orders[filled_filter]
stock_orders = stock_orders.iloc[::-1]
stock_orders['Total Cost'] = stock_orders['Total Qty'] * stock_orders['Avg Price']
stock_orders = update_cost_quantity(stock_orders)

In [749]:
def shares_owned(df):
    counts = {}
    symbols = df['Symbol'].unique()
    for symbol in symbols:
        counts[symbol] = [0.0, 0.0]
    for index, row in df.iterrows():
        symbol = row['Symbol']
        counts[symbol][0] += row['Total Qty']
        counts[symbol][1] += row['Total Cost']
        df.loc[index, 'Shares Owned'] = counts[symbol][0]
        if row.Side == 'Sell': 
            df.loc[index, 'P/L'] = counts[symbol][1]
        else:
            df.loc[index, 'P/L'] = '0'
    return df

In [750]:
stock_orders = shares_owned(stock_orders)

In [751]:
stock_orders

,Name,Symbol,Side,Status,Filled,Total Qty,Price,Avg Price,Time-in-Force,Placed Time,Filled Time,Total Cost,Shares Owned,P/L
57,Banco Santander,SAN,Sell,Filled,1.000000,-1.000000,@2.15,2.1500,DAY,04/23/2020 10:40:56 EDT,04/23/2020 10:46:08 EDT,2.1500,-1.000000,2.15
56,Snap,SNAP,Sell,Filled,1.000000,-1.000000,@17.58,17.5800,DAY,04/30/2020 08:57:33 EDT,04/30/2020 09:30:01 EDT,17.5800,-1.000000,17.58
52,Trxade Health Inc,MEDS,Buy,Filled,75.000000,75.000000,@6.88,6.8800,DAY,06/10/2021 15:25:14 EDT,06/10/2021 15:26:24 EDT,-516.0000,75.000000,0
49,Trxade Health Inc,MEDS,Sell,Filled,75.000000,-75.000000,@5.49,5.4900,DAY,06/11/2021 10:24:09 EDT,06/11/2021 10:24:09 EDT,411.7500,0.000000,-104.25
47,ADT,ADT,Sell,Filled,7.000000,-7.000000,@11.61,11.6100,DAY,06/14/2021 15:55:16 EDT,06/14/2021 15:56:10 EDT,81.2700,-7.000000,81.27
46,Zynga,ZNGA,Sell,Filled,1.000000,-1.000000,@10.65,10.6500,DAY,06/14/2021 15:55:47 EDT,06/14/2021 15:59:52 EDT,10.6500,-1.000000,10.65
45,Swestn Energy,SWN,Sell,Filled,1.000000,-1.000000,@5.48,5.4800,DAY,06/15/2021 09:41:22 EDT,06/15/2021 09:41:24 EDT,5.4800,-1.000000,5.48
44,GE,GE,Sell,Filled,1.000000,-1.000000,@13.42,13.4200,DAY,06/15/2021 09:41:44 EDT,06/15/2021 09:41:45 EDT,13.4200,-1.000000,13.42
43,Genworth Fincl,GNW,Sell,Filled,1.000000,-1.000000,@3.77,3.7700,DAY,07/07/2021 09:44:34 EDT,07/07/2021 09:46:10 EDT,3.7700,-1.000000,3.77
42,Antero Resources,AR,Sell,Filled,1.000000,-1.000000,@14.74,14.7400,DAY,07/09/2021 11:02:49 EDT,07/09/2021 11:02:50 EDT,14.7400,-1.000000,14.74


In [752]:
stock_symbols = stock_orders.groupby('Symbol')
stock_metrics = {}
for symbol, group in stock_symbols:
    totals = {}
    totals['P/L'] = 0.0
    for index, row in group.iterrows():
        totals['P/L'] = totals['P/L'] + float(row['P/L'])
    stock_metrics[symbol] = totals
print(stock_metrics)

{'ADGI': {'P/L': -34.0}, 'ADT': {'P/L': 81.27}, 'AHI': {'P/L': -75.05000000000001}, 'AM': {'P/L': 10.44}, 'AR': {'P/L': 14.74}, 'BBIG': {'P/L': -49.94999999999999}, 'CASI': {'P/L': 14.520000000000095}, 'CEI': {'P/L': 810.9500000000006}, 'CLNN': {'P/L': -19.72}, 'CLVS': {'P/L': 0.0}, 'CX': {'P/L': 16.66}, 'GE': {'P/L': 13.42}, 'GNW': {'P/L': 14.48}, 'MEDS': {'P/L': -104.25}, 'OPK': {'P/L': 3.56}, 'PETV': {'P/L': 50.099999999999966}, 'SAN': {'P/L': 2.15}, 'SHIBUSD': {'P/L': 0.0}, 'SNAP': {'P/L': 17.58}, 'SNDL': {'P/L': -14.055599999999913}, 'SWN': {'P/L': 5.48}, 'WISH': {'P/L': -10.150000000000006}, 'ZNGA': {'P/L': 10.65}}
